In [1]:
import torch
import torchphysics as tp
import pytorch_lightning as pl
import itertools
from torchphysics.problem.domains.domain3D.trimesh_polyhedron import TrimeshPolyhedron
#import os
#os.environ["CUDA_VISIBLE_DEVICES"] = "3"

In [2]:
# Spaces for in- and output
X = tp.spaces.R1('x')*tp.spaces.R1('y')*tp.spaces.R1('z')
U = tp.spaces.R3('u')

# Parameter
kappa = 18.3333 # GPa = 18333,33 Mpa
mu = 3.92857 # GPa = 3928.57 Mpa
u_dis = 0.2 # mm

In [ ]:
# Load stl.file and create domain
p = TrimeshPolyhedron(X, file_name='./data/geom_mech.stl')

p_slice = p.slice_with_plane(X['x', 'y']) # 2D domain
height_interval = tp.domains.Interval(tp.spaces.R1('z'), 0.0, 1.0)

# filter functions for boundary points
radius_s = 0.75**2
tol = 0.005 # tolerance for points at hole
tol_z = 1.e-4 # tolerance for bottom and top surface
def filter_top_hole(x, y):
    return (x + 12.67)**2 + (y - 3.5)**2 <= radius_s + tol

def filter_bottom_hole(x, y):
    return (x + 12.67)**2 + (y - 0.2)**2 <= radius_s + tol

def filter_top_bottom(z):
    return torch.logical_or(z < tol_z, z > 1 - tol_z)

def stress_surface(x, y, z):
    on_z = filter_top_bottom(z)
    in_hole = torch.logical_or(filter_top_hole(x, y), filter_bottom_hole(x, y))
    return torch.logical_not(torch.logical_or(on_z, in_hole))


In [ ]:
# network to train
model = tp.models.Sequential(
    tp.models.NormalizationLayer(p),
    tp.models.FCN(input_space=X, output_space=U, hidden=(30,30,30)))

In [ ]:
inner_sampler = tp.samplers.RandomUniformSampler(p, n_points = 18000).make_static() 
top_hole_sampler = tp.samplers.RandomUniformSampler(p.boundary, n_points=2000, 
                                                    filter_fn=filter_top_hole).make_static()
bottom_hole_sampler = tp.samplers.RandomUniformSampler(p.boundary, n_points=2000, 
                                                       filter_fn=filter_bottom_hole).make_static()
surface_sampler = tp.samplers.RandomUniformSampler(p.boundary, n_points=12000, 
                                                   filter_fn=stress_surface).make_static()
#fig = tp.utils.scatter(X, surface_sampler, top_hole_sampler, bottom_hole_sampler)
#fig.get_axes()[0].view_init(90, 270)
#fig.set_size_inches(10, 8)

In [ ]:
# Compute C:
def delta(a, b):
    return a == b

C = torch.zeros((3, 3, 3, 3), dtype=torch.float32)
for i,j,k,l in itertools.product(range(3), range(3), range(3), range(3)):
    unit_prod = delta(i, j) * delta(k, l)
    P_sym = 0.5 * (delta(i, k) * delta(j, l) + delta(j, k) * delta(i, l)) - 1/3.0*unit_prod
    C[i,j,k,l] = kappa * unit_prod + 2*mu*P_sym

# reshape to (9, 1, 3, 3), to apply to batch of the form ("1", batch_dim, 3, 3)
C = C.reshape(9, 1, 3, 3)

In [ ]:
## test C stuff:
#a = torch.tensor([[[1.0, 0.0, 2.0], [1.0, 1.0, 2.0], [0.0, 1.0, 2.0]], 
#                  [[1.0, 1.0, 2.0], [1.0, 5.0, 2.0], [5.0, 1.0, 2.0]]])
#b = torch.matmul(C, a)
#print(b)
#print(b.sum(dim=(2, 3)).T.reshape(len(a), 3, 3))

In [ ]:
# inner pde condition:
def pde_residual(u, C, x, y, z):
    sym_grad = tp.utils.sym_grad(u, x, y, z)
    sigma = torch.matmul(C, sym_grad).sum(dim=(2, 3)).T.reshape(len(u), 3, 3)
    return tp.utils.matrix_div(sigma, x, y, z) # + forces


pde_condition = tp.conditions.PINNCondition(module=model,
                                            sampler=inner_sampler,
                                            residual_fn=pde_residual, 
                                            data_functions={'C': C})

In [ ]:
# boundary conditions
# normal strain at 'outer' boundary = 0
def strain_residual(u, x, y, z, n, C):
    sym_grad = tp.utils.sym_grad(u, x, y, z)
    sigma = torch.matmul(C, sym_grad).sum(dim=(2, 3)).T.reshape(len(u), 3, 3)
    return torch.matmul(sigma, n)

def normal_fn(x, y, z):
    return p.boundary.normal(torch.cat((x,y,z), dim=-1)).unsqueeze(2) # for correct shape 

strain_condition = tp.conditions.PINNCondition(module=model,
                                               sampler=surface_sampler,
                                               residual_fn=strain_residual, 
                                               data_functions={'n': normal_fn, 'C': C})

In [ ]:
# periodic in z
side_sampler = tp.samplers.RandomUniformSampler(p_slice, n_points=6000).make_static()

def bc_fun(u_left, u_right):
    return u_left - u_right

periodic_cond = tp.conditions.PeriodicCondition(module=model,
                                                periodic_interval=height_interval,
                                                non_periodic_sampler=side_sampler,
                                                residual_fn=bc_fun, 
                                                weight=10.0)

In [ ]:
# displacement:
def displacement_residual(u):
    return u - torch.tensor([0.0, u_dis, 0.0], device=u.device)

displacement_condition = tp.conditions.PINNCondition(module=model,
                                                     sampler=top_hole_sampler,
                                                     residual_fn=displacement_residual,
                                                     weight=100)

# fixed at lower hole
def fix_residual(u):
    return u

fix_condition = tp.conditions.PINNCondition(module=model,
                                            sampler=bottom_hole_sampler,
                                            residual_fn=fix_residual, 
                                            weight=100)

In [ ]:
solver = tp.solver.Solver([strain_condition, fix_condition, displacement_condition, 
                           pde_condition, periodic_cond])

trainer = pl.Trainer(gpus='-1' if torch.cuda.is_available() else None,
                     num_sanity_val_steps=0,
                     benchmark=True,
                     max_steps=12000,
                     logger=False,
                     checkpoint_callback=False
                     )

trainer.fit(solver)

In [ ]:
samp = tp.samplers.PlotSampler(p_slice, n_points=3000, data_for_other_variables={'z': 0.5})
fig = tp.utils.plot(model, lambda u: u[:, 1:2], samp, plot_type='contour_surface')

In [ ]:
def strain_fn(u, x, y, z):
    sym_grad = tp.utils.sym_grad(u, x, y, z)
    sigma = torch.matmul(C, sym_grad).sum(dim=(2, 3)).T.reshape(len(u), 3, 3)
    return torch.norm(sigma, dim=(1,2))
fig = tp.utils.plot(model, strain_fn, samp, plot_type='contour_surface')